In [1]:
# GenIce用に作った、cif読み込みルーチン pip install cif2ice
from cif2ice import read_cif

# CIFの読みこみの部分はPyCIFRWから流用
# MethaneARはセルの形や大きさをすこしいじってある。それにともない、CH距離や角度も狂っている。あとで補正する
atoms, cellmat = read_cif.read_and_process("MethaneAR.cif")
# CIFは必ずfractional coordinateで記述されている
atoms, cellmat, len(atoms)

([('H', 0.025000000000000355, 0.6750000000000007, 0.11800000000000033),
  ('H', 0.6750000000000007, 0.11800000000000033, 0.025000000000000355),
  ('H', 0.08999999999999986, 0.6189999999999998, 0.2759999999999998),
  ('H', 0.6189999999999998, 0.2759999999999998, 0.08999999999999986),
  ('H', 0.1869999999999994, 0.7289999999999992, 0.17099999999999937),
  ('H', 0.7289999999999992, 0.17099999999999937, 0.1869999999999994),
  ('H', 0.0389999999999997, 0.7899999999999991, 0.2530000000000001),
  ('H', 0.7899999999999991, 0.2530000000000001, 0.0389999999999997),
  ('H', 0.006000000000000227, 0.3509999999999991, 0.9890000000000008),
  ('H', 0.3509999999999991, 0.9890000000000008, 0.006000000000000227),
  ('H', 0.07900000000000063, 0.3990000000000009, 0.8320000000000007),
  ('H', 0.3990000000000009, 0.8320000000000007, 0.07900000000000063),
  ('H', 0.031000000000000583, 0.5240000000000009, 0.9510000000000005),
  ('H', 0.5240000000000009, 0.9510000000000005, 0.031000000000000583),
  ('H', 0.1720

原子の順番がソートされてしまっているので、炭素と水素をくっつける作業が必要。

これが自動化できない原因か。CIFはX線で得られた構造の情報で、分子とか結合とか知ったことじゃないので、順番に意味がない。でもGromacsではTopologyファイルで定義される順番が重要。

鏡像になっても動いてくれるはず。

In [2]:
import numpy as np
# 炭素のみ抽出
carbons = np.array([atom[1:4] for atom in atoms if atom[0] == "C"])
carbons

array([[0.085  , 0.703  , 0.205  ],
       [0.703  , 0.205  , 0.085  ],
       [0.072  , 0.423  , 0.939  ],
       [0.423  , 0.939  , 0.072  ],
       [0.189  , 0.435  , 0.519  ],
       [0.435  , 0.519  , 0.189  ],
       [0.626  , 0.953  , 0.72   ],
       [0.953  , 0.72   , 0.626  ],
       [0.318  , 0.839  , 0.4454 ],
       [0.839  , 0.4454 , 0.318  ],
       [0.291  , 0.701  , 0.834  ],
       [0.701  , 0.834  , 0.291  ],
       [0.205  , 0.085  , 0.703  ],
       [0.939  , 0.072  , 0.423  ],
       [0.519  , 0.189  , 0.435  ],
       [0.72   , 0.626  , 0.953  ],
       [0.4454 , 0.318  , 0.839  ],
       [0.834  , 0.291  , 0.701  ],
       [0.187  , 0.187  , 0.187  ],
       [0.954  , 0.954  , 0.954  ],
       [0.57578, 0.57577, 0.57578]])

In [3]:
# 水素のみ抽出
hydrogens = np.array([atom[1:4] for atom in atoms if atom[0] == "H"])
hydrogens

array([[0.025, 0.675, 0.118],
       [0.675, 0.118, 0.025],
       [0.09 , 0.619, 0.276],
       [0.619, 0.276, 0.09 ],
       [0.187, 0.729, 0.171],
       [0.729, 0.171, 0.187],
       [0.039, 0.79 , 0.253],
       [0.79 , 0.253, 0.039],
       [0.006, 0.351, 0.989],
       [0.351, 0.989, 0.006],
       [0.079, 0.399, 0.832],
       [0.399, 0.832, 0.079],
       [0.031, 0.524, 0.951],
       [0.524, 0.951, 0.031],
       [0.172, 0.416, 0.983],
       [0.416, 0.983, 0.172],
       [0.235, 0.44 , 0.619],
       [0.44 , 0.619, 0.235],
       [0.1  , 0.372, 0.524],
       [0.372, 0.524, 0.1  ],
       [0.161, 0.536, 0.486],
       [0.536, 0.486, 0.161],
       [0.262, 0.392, 0.448],
       [0.392, 0.448, 0.262],
       [0.624, 0.895, 0.814],
       [0.895, 0.814, 0.624],
       [0.644, 0.885, 0.636],
       [0.885, 0.636, 0.644],
       [0.53 , 0.005, 0.707],
       [0.005, 0.707, 0.53 ],
       [0.707, 0.026, 0.724],
       [0.026, 0.724, 0.707],
       [0.279, 0.741, 0.473],
       [0.

In [4]:
len(carbons), len(hydrogens)

(21, 84)

In [5]:

from genice2.cell import cellvectors

# cellは単位胞の辺の長さと相互の角度で記述される。これを行列(横ベクトル形式)に変換する
cellmat = cellvectors(*cellmat)
# CIFの長さはÅ、でもgromacsはnm
cellmat /= 10  # angstrom to nm


炭素に直結する水素を選ぶ。

In [6]:
neighbors = []
for carbon in carbons:
    # C-H ベクトル
    CH = hydrogens - carbon
    # fractional coordinateでの周期境界処理
    CH -= np.floor(CH+0.5)
    # 実座標に変換
    CH = CH @ cellmat
    # 個々のベクトルの長さ
    CHlen = np.linalg.norm(CH, axis=1)
    # ベクトル長が1未満の要素番号を得る
    neighbors.append(np.argwhere(CHlen < 0.16).reshape(-1))

neighbors

[array([0, 2, 4, 6]),
 array([1, 3, 5, 7]),
 array([ 8, 10, 12, 14]),
 array([ 9, 11, 13, 15]),
 array([16, 18, 20, 22]),
 array([17, 19, 21, 23]),
 array([24, 26, 28, 30]),
 array([25, 27, 29, 31]),
 array([32, 34, 36, 38]),
 array([33, 35, 37, 39]),
 array([40, 42, 44, 46]),
 array([41, 43, 45, 47]),
 array([54, 55, 56, 57]),
 array([58, 59, 60, 61]),
 array([62, 63, 64, 65]),
 array([66, 67, 68, 69]),
 array([70, 71, 72, 73]),
 array([74, 75, 76, 77]),
 array([48, 49, 78, 79]),
 array([50, 51, 80, 81]),
 array([52, 53, 82, 83])]

In [7]:
import gromacs

CH = 0.1087  # nm
H4 = (
    np.array([[-1, -1, -1], [-1, +1, +1], [+1, -1, +1], [+1, +1, -1]]) / 3**0.5 * CH
)


def organize(carbon, hydrogens, cellmat):
    # 1. estimate the axes.
    dh = hydrogens - carbon
    dh -= np.floor(dh+0.5)
    vecs = dh @ cellmat
    x = (vecs[2]+vecs[3]) - (vecs[0]+vecs[1])
    y = (vecs[1]+vecs[3]) - (vecs[0]+vecs[2])
    z = (vecs[1]+vecs[2]) - (vecs[0]+vecs[3])
    x /= np.linalg.norm(x)
    y /= np.linalg.norm(y)
    z /= np.linalg.norm(z)
    print(x@y,y@z,z@x)
    a = np.vstack([x,y,z])
    # 特異値分解により直交させる。
    U, S, Vt = np.linalg.svd(a, full_matrices=True)
    a = U@Vt
    return H4 @ a + carbon @ cellmat


# # 全原子を絶対座標にする
# carbons = carbons @ cellmat
# hydrogens = hydrogens @ cellmat

# 炭素、水素1、水素2、、の順番に並べかえる。
atom = []
positions = []
for carbon, neis in zip(carbons, neighbors):
    atom.append("C")
    positions.append(carbon @ cellmat)
    
    # ここで水素の配向をととのえる。
    newpos = organize(carbon, hydrogens[neis], cellmat)
    for h in newpos:
        atom.append("H")
        positions.append(h)
        

# その他の情報はその場で合成する
residue = ["MET" for _ in atom]
resi_id = [(i//5)+1 for i in range(len(atom))]
atom_id = [i+1 for i in range(len(atom))]

# 1フレーム分のデータを辞書に入れる
frame = {
    "residue": residue,
    "atom": atom,
    "position": positions,
    "resi_id": resi_id,
    "atom_id": atom_id,
    "cell": cellmat,
}

# 書きだし
with open("MethaneAR.gro", "w") as f:
    gromacs.write_gro(frame, f)

-0.013836273371864438 -0.000371093073652567 -0.006248241369162064
-0.013836273371864449 -0.0003710930736525346 -0.00624824136916207
-0.00427248167404398 -0.004425544495760891 0.004064019517894646
-0.004272481674043998 -0.004425544495760877 0.0040640195178945964
0.0033109142956036894 -0.007890644055311851 -0.0006018430228045446
0.003310914295603703 -0.007890644055311811 -0.0006018430228045343
-0.0044436558067479795 0.0019458244230511105 0.012363610465808536
-0.00444365580674799 0.0019458244230510721 0.012363610465808533
-0.01246701391042824 0.008038287779830282 -0.0039853288566024945
-0.012467013910428223 0.00803828777983028 -0.00398532885660253
0.010963796385876797 -0.0008701471682781607 -0.005216053762719921
0.010963796385876794 -0.0008701471682781221 -0.005216053762719933
-0.013836273371864447 -0.00037109307365257814 -0.006248241369162085
-0.004272481674044002 -0.0044255444957608265 0.004064019517894658
0.0033109142956037167 -0.007890644055311834 -0.0006018430228045645
-0.00444365580

In [8]:
# groファイルを読みこんでセルを4x4x4倍にするだけ

import gromacs
import numpy as np

with open("MethaneAR.gro") as f:
    # read_groはiterator。今は最初のフレームだけ欲しいのでbreakする。
    for frame in gromacs.read_gro(f):
        break

# 繰り返しの数。
repeat = (3, 3, 3)
# セル行列
cellmat = frame["cell"]
# 逆行列
celli = np.linalg.inv(cellmat)
# 相対座標になおしてから、縮小する。
unitcell = frame["position"] @ celli / np.array(repeat)

# 相対座標をコピペして64個並べる
positions = [
    unitcell + np.array([x / repeat[0], y / repeat[1], z/repeat[2]])
    for x in range(repeat[0])
    for y in range(repeat[1])
    for z in range(repeat[2])
]

# 拡大後の原子数
numAtoms = len(unitcell) * repeat[0] * repeat[1] * repeat[2]

# セルを大きくする。
cellmat *= np.array(repeat)
# 1フレーム分を準備
frame = {
    # 絶対座標に戻す
    "position" : np.vstack(positions) @ cellmat,
    "resi_id": [(i//5)+1 for i in range(numAtoms)],
    "atom_id": [i+1 for i in range(numAtoms)],
    "atom": frame["atom"] * repeat[0] * repeat[1] * repeat[2],
    "residue": frame["residue"] * repeat[0] * repeat[1] * repeat[2],
    "cell": cellmat
}

# 書き出す。
with open("MethaneAR333.gro", "w") as f:
    gromacs.write_gro(frame, f)


